In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os, glob
import sys
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from gpu_starter_mirror_strategy import start_gpus
from sklearn.model_selection import train_test_split

2023-04-17 22:45:05.597017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 22:45:05.694125: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-17 22:45:05.721884: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-17 22:45:06.162392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
strategy, REPLICAS, AUTO = start_gpus([0])

2023-04-17 22:45:07.941311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 22:45:07.970386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 22:45:07.971795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 22:45:07.973832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Returning objects as strategy, replicas and auto in same order.


In [3]:
train_files = glob.glob('../data/train/*.jpg')

In [4]:
labels = []
for x in train_files:
    # dog = 1
    # cat = 0
    if 'dog' in x:
        labels.append(1)
    else:
        labels.append(0)

In [5]:
train_data = [(train_files[x], labels[x]) for x in range(len(train_files))]

In [6]:
def split_data(data_files):
    train, test = train_test_split(data_files, test_size=0.05, random_state=42)
    train, val = train_test_split(train, test_size=0.05, random_state=42)
    return train, val, test

In [7]:
train, val, test = split_data(train_data)

In [8]:
print(f'Train: {len(train)}')
print(f'Val: {len(val)}')
print(f'Test: {len(test)}')

Train: 22562
Val: 1188
Test: 1250


In [21]:
def read_img(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, size = (256, 256))
    img = img / 255
    return img, label

In [93]:
def get_data(data, repe = True):
    img, label = [x[0] for x in data], [x[1] for x in data]
    tensor = tf.data.Dataset.from_tensor_slices((img, label))
    if repe:
        tensor = tensor.repeat()
    tensor = tensor.map(read_img)
    tensor = tensor.batch(32)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [94]:
train_dataset = get_data(train)
val_dataset = get_data(val, repe=False)

In [ ]:
def create_model(model_name, shape):
    with strategy.scope():
        input_layer = tf.keras.Input(shape = shape)
        construct = getattr(keras.applications, model_name)
        mid_layer = construct(include_top = False, 
            weights = None, 
            pooling = 'avg')(input_layer)
        last_layer = keras.layers.Dense(1, activation = 'sigmoid')(mid_layer)
        model = keras.Model(input_layer, last_layer)
    return model
def compile_new_model(model):
    with strategy.scope():
        loss = keras.losses.BinaryCrossentropy()
        optimizer = keras.optimizers.SGD()
        prec = keras.metrics.Precision(name = 'prec')
        rec = keras.metrics.Recall(name = 'rec')
        model.compile(
            loss = loss,
            optimizer = optimizer,
            metrics = [prec, rec]
        )
    return model

In [95]:
def make_model():
    input_layer = keras.Input(shape = (256, 256, 3))
    res_layer = keras.applications.ResNet50(
        input_shape = (256, 256, 3),
        include_top=False,
        pooling = 'avg'
        )(input_layer)
    dense_layer = keras.layers.Dense(1, activation = 'sigmoid')(res_layer)
    model = keras.Model(input_layer, dense_layer)
    return model
def compile_model():
    with strategy.scope():
        model = make_model()
        loss = keras.losses.BinaryCrossentropy()
        optim = keras.optimizers.SGD()
        model.compile(loss=loss, optimizer=optim)
    return model

In [96]:
model = compile_model()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [97]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_12 (Dense)            (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________


In [99]:
model_hist = model.fit(
    train_dataset,
    epochs = 5,
    steps_per_epoch = (len(train) // 32),
    validation_data = val_dataset,
    verbose = 1
)

Epoch 1/5


2023-04-17 23:29:02.266943: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 22562
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:47"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
   

705/705 [==============================] - ETA: 0s - loss: 0.0652

2023-04-17 23:31:06.319498: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1188
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:52"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
    

705/705 [==============================] - 127s 170ms/step - loss: 0.0652 - val_loss: 0.3056
Epoch 2/5
705/705 [==============================] - 119s 169ms/step - loss: 0.0113 - val_loss: 0.0178
Epoch 3/5
705/705 [==============================] - 119s 169ms/step - loss: 0.0038 - val_loss: 0.0182
Epoch 4/5
705/705 [==============================] - 119s 169ms/step - loss: 0.0025 - val_loss: 0.0166
Epoch 5/5
259/705 [==========>...................] - ETA: 1:14 - loss: 0.0015